In [1]:
!pip install git+https://github.com/business-science/pymodeltime.git

  Cloning https://github.com/business-science/pymodeltime.git to /tmp/pip-req-build-2kbc0_e0
  Running command git clone --filter=blob:none --quiet https://github.com/business-science/pymodeltime.git /tmp/pip-req-build-2kbc0_e0
  Resolved https://github.com/business-science/pymodeltime.git to commit f83d8bf6f5ba853d4ed14ed6886fb394a7244c93
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.2/265.2 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.7/416.7 kB 29.8 MB/s eta 0:00:00
     ━━━━━

In [3]:
import pytimetk as tk
from pymodeltime import MLModelWrapper
from pymodeltime import ArimaReg, ProphetReg
from pymodeltime import ModelTimeTable, ModelTimeAccuracy, ModelTimeCalibration, ModelTimeForecast,ModelTimeRefit
from pymodeltime import ArimaReg, ProphetReg
from pymodeltime import MLModelWrapper,H2OAutoMLWrapper
from pymodeltime import plot_modeltime_forecast
from pymodeltime import plot_modeltime_forecast_2

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
# Load data
import pandas as pd
import pandas as pd
import pytimetk as tk
from sklearn.model_selection import train_test_split
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

import logging

# Set logging level to WARNING to suppress DEBUG and INFO messages
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)


In [4]:
# We start by loading the dataset
# /walmart_sales_weekly.html
dset = tk.load_dataset('walmart_sales_weekly', parse_dates = ['Date'])
# Rename the 'Date' column to 'date'
dset = dset.rename(columns={'Date': 'date'})

dset = dset.drop(columns=[
    'id', # This column can be removed as it is equivalent to 'Dept'
    'Store', # This column has only one possible value
    'Type', # This column has only one possible value
    'Size', # This column has only one possible value
    'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5',
    'IsHoliday', 'Temperature', 'Fuel_Price', 'CPI',
       'Unemployment'])

dset['date'] = pd.to_datetime(dset['date'])
# Get the maximum (last) date in your dataset
max_date = dset['date'].max()

# Create a DataFrame with only the last date
last_date_df = pd.DataFrame({'date': [max_date]})

future_dates = dset.groupby('Dept').future_frame(
    date_column='date',
    length_out=5
)
# Append these future dates to your original data
extended_data = pd.concat([dset, future_dates]).reset_index(drop=True)
sales_df_dates = extended_data.augment_timeseries_signature(date_column = 'date')
sales_df_dates.head(10)

sales_df_dates = sales_df_dates[[
    'date'
    ,'Dept'
    , 'Weekly_Sales'
    , 'date_year'
    , 'date_month'
    , 'date_yweek'
    , 'date_mweek'
    ]]
sales_df_dates.tail(10)
df_with_lags = sales_df_dates \
    .groupby('Dept') \
    .augment_lags(
        date_column  = 'date',
        value_column = 'Weekly_Sales',
        lags         = [5,6,7,8,9]
    )
df_with_lags.head(5)

lag_columns = [col for col in df_with_lags.columns if 'lag' in col]

df_with_rolling = df_with_lags \
    .groupby('Dept') \
    .augment_rolling(
        date_column  = 'date',
        value_column = lag_columns,
        window  = 4,
        window_func = 'mean',
        threads = 1 # Change to -1 to use all available cores
    )
df_with_rolling[df_with_rolling.Dept ==1].head(10)

all_lag_columns = [col for col in df_with_rolling.columns if 'lag' in col]

df_no_nas = df_with_rolling \
    .dropna(subset=all_lag_columns, inplace=False)

df_no_nas.head()

future = df_no_nas[df_no_nas.Weekly_Sales.isnull()]
train = df_no_nas[df_no_nas.Weekly_Sales.notnull()]

train.to_csv("./train.csv",index = False)

import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('./train.csv')

data['date'] = pd.to_datetime(data['date'])

# Assuming 'data' is your initial DataFrame
all_columns = train.columns.tolist()
excluded_columns = ['date', 'Weekly_Sales', 'Dept']  # Columns to exclude
columns_to_convert = [col for col in all_columns if col not in excluded_columns]

# Convert columns in the original DataFrame
data.loc[:, columns_to_convert] = data.loc[:, columns_to_convert].astype('float32')

# Splitting the data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)


test_data.loc[:, columns_to_convert] = test_data[columns_to_convert].astype('float32')

test_data.head(2)

Future framing...:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Rolling...:   0%|          | 0/7 [00:00<?, ?it/s]

,date,Dept,Weekly_Sales,date_year,date_month,date_yweek,date_mweek,Weekly_Sales_lag_5,Weekly_Sales_lag_6,Weekly_Sales_lag_7,Weekly_Sales_lag_8,Weekly_Sales_lag_9,Weekly_Sales_lag_5_rolling_mean_win_4,Weekly_Sales_lag_6_rolling_mean_win_4,Weekly_Sales_lag_7_rolling_mean_win_4,Weekly_Sales_lag_8_rolling_mean_win_4,Weekly_Sales_lag_9_rolling_mean_win_4
7672,2010-09-24,38,87720.98,2010,9,38,4,78481.320312,90990.929688,90990.929688,77852.742188,77852.742188,84578.984375,84421.835938,79248.90625,74075.976562,74341.6875
7673,2010-09-24,38,87720.98,2010,9,38,4,78481.320312,90990.929688,90990.929688,77852.742188,77852.742188,84578.984375,84421.835938,79248.90625,74075.976562,74341.6875


In [5]:
# Define the target column
target_column = 'Weekly_Sales'

# Select feature columns - all columns except 'date', 'value', and 'id'
feature_columns = [col for col in train_data.columns if col not in ['date', target_column, 'Dept']]

# Extract features (X) and target (y) from the training data
X_train = train_data[feature_columns]
y_train = train_data[target_column]

# Display the first few rows of X_train to verify
X_test = test_data[feature_columns]
y_test = test_data[target_column]

In [6]:

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Create instances of the ML models
rf_model = RandomForestRegressor()
ml_rf_wrapper = MLModelWrapper(rf_model, feature_columns, "Random Forest_sk")

xgb_model = XGBRegressor()
ml_xgb_wrapper = MLModelWrapper(xgb_model, feature_columns, "XGBoost_sk")

# Fit the models (assuming train_data and y_train are defined)
ml_rf_wrapper.fit(train_data, y_train)
ml_xgb_wrapper.fit(train_data, y_train)

# Make predictions with the Random Forest model
rf_predictions = ml_rf_wrapper.predict(test_data)

# Make predictions with the XGBoost model
xgb_predictions = ml_xgb_wrapper.predict(test_data)


# Example of evaluating the model performance (assuming you have y_test)
from sklearn.metrics import mean_squared_error

# Assuming y_test contains the true target values for the test set

# Calculate Mean Squared Error for Random Forest model
rf_mse = mean_squared_error(y_test, rf_predictions)

# Calculate Mean Squared Error for XGBoost model
xgb_mse = mean_squared_error(y_test, xgb_predictions)

print(f"Random Forest MSE: {rf_mse}")
print(f"XGBoost MSE: {xgb_mse}")


Random Forest MSE: 42260462.3907543
XGBoost MSE: 47919517.56054897


In [7]:
import h2o
from h2o.automl import H2OAutoML


import h2o
from h2o.automl import H2OAutoML
# from pymodeltime import H2OAutoMLWrapper

# Start H2O cluster
h2o.init()

# Convert your DataFrame to H2OFrame
h2o_train = h2o.H2OFrame(train_data)

# Initialize and train H2O AutoML model
automl = H2OAutoML(max_models= 2, seed=1, max_runtime_secs= 1000)
automl.train(x=feature_columns, y=target_column, training_frame=h2o_train)

# Get the best model
best_h2o_model = automl.leader

# Wrap the model with H2OAutoMLWrapper
h2o_automl_wrapper = H2OAutoMLWrapper(best_h2o_model, target_column=target_column)



Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.21" 2023-10-17; OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp45aiuuu0
  JVM stdout: /tmp/tmp45aiuuu0/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp45aiuuu0/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,16 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_unknownUser_0ekj0w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,11 mins 30 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_unknownUser_0ekj0w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.155 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [8]:
# Extract the leaderboard
lb = automl.leaderboard

# Optionally, you can convert the leaderboard to a pandas DataFrame for easier manipulation
lb_df = lb.as_data_frame()

# Display the leaderboard
print(lb_df.head())


Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
                                            model_id          rmse  \
0                 XGBoost_1_AutoML_1_20240103_155719      0.005838   
1  StackedEnsemble_BestOfFamily_1_AutoML_1_202401...      8.228178   
2                     GLM_1_AutoML_1_20240103_155719  32894.515264   

            mse           mae         rmsle  mean_residual_deviance  
0  3.407959e-05      0.003422  1.783788e-07            3.407959e-05  
1  6.770291e+01      6.490634  4.085326e-04            6.770291e+01  
2  1.082049e+09  25952.910617  8.109514e-01            1.082049e+09  


In [9]:

from pymodeltime import AutoGluonTabularWrapper

auto_gluon_wrapper = AutoGluonTabularWrapper(target_column="Weekly_Sales")
auto_gluon_wrapper.fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20240103_160829"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20240103_160829"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 

[1000]	valid_set's rmse: 1354.36
[2000]	valid_set's rmse: 681.355
[3000]	valid_set's rmse: 412.009
[4000]	valid_set's rmse: 273.09
[5000]	valid_set's rmse: 195.288
[6000]	valid_set's rmse: 144.131
[7000]	valid_set's rmse: 110.474
[8000]	valid_set's rmse: 85.9228
[9000]	valid_set's rmse: 67.1298
[10000]	valid_set's rmse: 53.4679


	-53.4679	 = Validation score   (-root_mean_squared_error)
	17.46s	 = Training   runtime
	1.94s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 3572.24s of the 3572.22s of remaining time.


[1000]	valid_set's rmse: 173.122
[2000]	valid_set's rmse: 30.9376
[3000]	valid_set's rmse: 7.83257
[4000]	valid_set's rmse: 2.00746
[5000]	valid_set's rmse: 0.531094
[6000]	valid_set's rmse: 0.149888
[7000]	valid_set's rmse: 0.0444413
[8000]	valid_set's rmse: 0.0137766
[9000]	valid_set's rmse: 0.004021
[10000]	valid_set's rmse: 0.00124684


	-0.002	 = Validation score   (-root_mean_squared_error)
	19.04s	 = Training   runtime
	1.6s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 3548.81s of the 3548.79s of remaining time.
	-220.5605	 = Validation score   (-root_mean_squared_error)
	11.52s	 = Training   runtime
	0.18s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 3536.95s of the 3536.93s of remaining time.
	-2.2493	 = Validation score   (-root_mean_squared_error)
	157.31s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ... Training model for up to 3379.55s of the 3379.54s of remaining time.
	-179.8107	 = Validation score   (-root_mean_squared_error)
	5.41s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetFastAI ... Training model for up to 3373.87s of the 3373.86s of remaining time.
	-4899.1518	 = Validation score   (-root_mean_squared_error)
	10.7s	 = Training   runtime
	0.04s	 = Validation runtime
Fitt

[1000]	valid_set's rmse: 4.49107
[2000]	valid_set's rmse: 0.0362057
[3000]	valid_set's rmse: 0.000323703
[4000]	valid_set's rmse: 3.02701e-06
[5000]	valid_set's rmse: 2.78666e-08
[6000]	valid_set's rmse: 6.1078e-10
[7000]	valid_set's rmse: 2.96926e-10
[8000]	valid_set's rmse: 2.88733e-10
[9000]	valid_set's rmse: 2.86819e-10
[10000]	valid_set's rmse: 2.85468e-10


	-0.0015	 = Validation score   (-root_mean_squared_error)
	60.91s	 = Training   runtime
	3.91s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the 3183.05s of remaining time.
	Ensemble Weights: {'LightGBMLarge': 1.0}
	-0.0015	 = Validation score   (-root_mean_squared_error)
	0.45s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 417.47s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20240103_160829")


Calibration completed for AutoGluonTabular model.


In [10]:


from ipywidgets import interact
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import pandas as pd

modeltime_table = ModelTimeTable(ml_rf_wrapper,h2o_automl_wrapper,auto_gluon_wrapper

                                 )

model_time_calibrator = ModelTimeCalibration(modeltime_table, test_data,target_column = 'Weekly_Sales')
model_time_calibrator.calibrate()
calibration_results_df = model_time_calibrator.get_calibration_results()

print(calibration_results_df.to_string(index=False))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
Calibration completed for AutoGluonTabular model.
 .model_id   .model                        .model_desc    .type   .calibration_data
         1 <fit[+]>                   Random Forest_sk ML Model <tibble [1918 × 4]>
         2 <fit[+]> XGBoost_1_AutoML_1_20240103_155719   Custom <tibble [1918 × 4]>
         3 <fit[+]>                WeightedEnsemble_L2   Custom <tibble [1918 × 4]>


In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np


modeltime_accuracy = ModelTimeAccuracy(modeltime_table, test_data, target_column='Weekly_Sales')
accuracy_results = modeltime_accuracy.calculate_accuracy()
print(accuracy_results.to_string(index=False))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
 model_id                         model_desc         mae        rmse       r2
        1                   Random Forest_sk 4771.767897 6500.804749 0.930775
        2 XGBoost_1_AutoML_1_20240103_155719 4992.283275 6576.087468 0.929163
        3                WeightedEnsemble_L2 4046.045267 5608.922419 0.948467


In [13]:
from pymodeltime import ModelTimeForecast

forecast_test = ModelTimeForecast(modeltime_table, new_data=test_data,
                                  actual_data=data,target_column= target_column)
forecast_results_test = forecast_test.forecast()
forecast_results_test.to_csv("forecast_results_test.csv", index=False)

Processing predictions for model: MLModelWrapper and department: 1
Processing predictions for model: MLModelWrapper and department: 1
No data available for predictions for department: 1
Processing predictions for model: H2OAutoMLWrapper and department: 1
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,19 mins 38 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_unknownUser_0ekj0w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.155 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Processing predictions for model: AutoGluonTabularWrapper and department: 1
Processing predictions for model: MLModelWrapper and department: 3
Processing predictions for model: MLModelWrapper and department: 3
No data available for predictions for department: 3
Processing predictions for model: H2OAutoMLWrapper and department: 3
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,19 mins 39 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_unknownUser_0ekj0w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.155 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Processing predictions for model: AutoGluonTabularWrapper and department: 3
Processing predictions for model: MLModelWrapper and department: 8
Processing predictions for model: MLModelWrapper and department: 8
No data available for predictions for department: 8
Processing predictions for model: H2OAutoMLWrapper and department: 8
Checking whether there is an H2O instance running at http://localhost:54321. connected.


...

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
Processing predictions for model: AutoGluonTabularWrapper and department: 38
Processing predictions for model: MLModelWrapper and department: 93
Processing predictions for model: MLModelWrapper and department: 93
Processing predictions for model: H2OAutoMLWrapper and department: 93
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,19 mins 51 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_unknownUser_0ekj0w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.155 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
Processing predictions for model: AutoGluonTabularWrapper and department: 93
Processing predictions for model: MLModelWrapper and department: 95
Processing predictions for model: MLModelWrapper and department: 95
Processing predictions for model: H2OAutoMLWrapper and department: 95
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,20 mins 04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_unknownUser_0ekj0w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.155 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
Processing predictions for model: AutoGluonTabularWrapper and department: 95


In [15]:


plot_modeltime_forecast_2(forecast_results_test, multi_column_display=True)


In [16]:

from concurrent.futures import ThreadPoolExecutor
import numpy as np
import pandas as pd
import h2o
from pymodeltime import AutoGluonTabularWrapper

from pymodeltime import ModelTimeRefit
from h2o.automl import H2OAutoML
# Initialize the refitter
refitter = ModelTimeRefit(modeltime_table, verbose=True, parallel=True, max_workers=4)

# Refit the models with the new data
new_data = pd.concat([train_data, test_data], ignore_index=True)  # Replace with your entire dataset (train + test)


# Convert only the numeric columns to float32
new_data[feature_columns] = new_data[feature_columns].astype('float32')

refitted_models = refitter.refit_models(new_data, target_column=target_column)

# Generate forecasts using the refitted models
forecast_results_refit = refitter.forecast(new_data, target_column=target_column)

# Concatenate all model forecasts into a single DataFrame
all_forecasts = pd.concat([result for result in forecast_results_refit.values() if result is not None], ignore_index=True)

# Save to CSV
all_forecasts.to_csv("./forecast_results_refit.csv", index=False)

Checking whether there is an H2O instance running at http://localhost:54321.

No path specified. Models will be saved in: "AutogluonModels/ag-20240103_161817"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20240103_161817"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 

 connected.


Fitting AutoMLPipelineFeatureGenerator...


H2O_cluster_uptime:,21 mins 19 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_unknownUser_0ekj0w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.155 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


...

[1000]	valid_set's rmse: 251.276
[2000]	valid_set's rmse: 62.5796
[3000]	valid_set's rmse: 19.0985
[4000]	valid_set's rmse: 6.50053
[5000]	valid_set's rmse: 2.18592
[6000]	valid_set's rmse: 0.727506
[7000]	valid_set's rmse: 0.246242
[8000]	valid_set's rmse: 0.0878782
[9000]	valid_set's rmse: 0.0317983
[10000]	valid_set's rmse: 0.0109251


	-0.0111	 = Validation score   (-root_mean_squared_error)
	19.89s	 = Training   runtime
	2.7s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-164.1371	 = Validation score   (-root_mean_squared_error)
	16.26s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: CatBoost ...
	-6.1367	 = Validation score   (-root_mean_squared_error)
	163.67s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-145.0091	 = Validation score   (-root_mean_squared_error)
	4.19s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-3701.4559	 = Validation score   (-root_mean_squared_error)
	13.03s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	-0.1539	 = Validation score   (-root_mean_squared_error)
	10.8s	 = Training   runtime
	0.36s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-730.784	 = Validation score   (-root_mean_squared_error)
	206.68s	 = Training   runtime
	0.01s	 = Va

[1000]	valid_set's rmse: 8.6863
[2000]	valid_set's rmse: 0.139636
[3000]	valid_set's rmse: 0.00216703
[4000]	valid_set's rmse: 4.23779e-05
[5000]	valid_set's rmse: 9.37456e-07
[6000]	valid_set's rmse: 2.2859e-08
[7000]	valid_set's rmse: 7.87438e-10
[8000]	valid_set's rmse: 5.97582e-10
[9000]	valid_set's rmse: 5.48835e-10
[10000]	valid_set's rmse: 5.41101e-10


	-0.0017	 = Validation score   (-root_mean_squared_error)
	64.89s	 = Training   runtime
	6.59s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'LightGBMLarge': 1.0}
	-0.0017	 = Validation score   (-root_mean_squared_error)
	0.35s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 543.17s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20240103_163553")
Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models ...
Fitting model: KNeighborsUnif_FULL ...
	0.02

Model successfully refitted: <pymodeltime.AutoGluonTabularWrapper.AutoGluonTabularWrapper object at 0x7bf380a342e0>
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%


In [17]:

forecast_future = ModelTimeForecast(refitted_models, actual_data=new_data, future_data=future, target_column=target_column)
forecast_results_future = forecast_future.forecast()
forecast_results_future.to_csv("forecast_results_future_mulit_refitted.csv", index=False)



Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,56 mins 08 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,14 days and 21 minutes
H2O_cluster_name:,H2O_from_python_unknownUser_0ekj0w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.081 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,56 mins 08 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,14 days and 21 minutes
H2O_cluster_name:,H2O_from_python_unknownUser_0ekj0w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.081 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
Checking whether there is an H2O instance running at http://localhost:54321. connected.


...

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,56 mins 12 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,14 days and 21 minutes
H2O_cluster_name:,H2O_from_python_unknownUser_0ekj0w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.081 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,56 mins 13 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,14 days and 21 minutes
H2O_cluster_name:,H2O_from_python_unknownUser_0ekj0w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.081 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%


In [18]:
plot_modeltime_forecast_2(forecast_results_future,  multi_column_display=True)